In [2]:
import io
import random
import string # to process standard python strings
import warnings
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


import pandas as pd

import re
import json
from collections import Counter
from matplotlib import pyplot as plt


import matplotlib.patches as patches
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords


rc('animation', html='jshtml')

stopwords = set(STOPWORDS)


In [3]:
nltk.download('popular', quiet=True)

True

In [4]:
f=open('/content/chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()
raw

'a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods. such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test. chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition. some chatterbots use sophisticated natural language processing systems, but many simpler systems scan for keywords within the input, then pull a reply with the most matching keywords, or the most similar wording pattern, from a database.\n\nthe term "chatterbot" was originally coined by michael mauldin (creator of the first verbot, julia) in 1994 to describe these conversational programs.today, most chatbots are either accessed via virtual assistants such as google assistant and amazon

In [5]:
raw ="""How dangerous is the coronavirus Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.How has the coronavirus affected you Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly. Both in respect of career and in respect of mental health. Is there a treatment for the Coronavirus They are experimenting with various medicines and it seems some anti-malarial drugs show promise. However, they help only a little bit, as evidenced by the mortality rate in places like Italy or Spain. They are best if administered early, before the lungs are damaged. Is there a vaccine for the coronavirus We don’t “find” vaccines. That’s the language yellow media uses, but it’s far from the truth.
We know the virus, and we know what it looks like. Now we need to produce a sufficient amount of attenuated virii. For that, we inject the virus into a foreign host, like a chicken embryo in an egg. Most viruses will die, because the host is so different from humans, that it can not work. Some will mutate to work, and those multiply.How did coronaviruses get their name The coronavirus family was discovered first in 1960s but we don’t know where they come from. They get the name given their crown-like (“corona”) shape. Following a December 2019 outbreak in China, the World Health Organization identified a new type called 2019 novel coronavirus (2019-nCoV).What are coronaviruses Coronavirus is a name for a group of viruses that target mammals, including cats, dogs, and humans. They’re well known for causing potentially fatal conditions like kidney failure, pneumonia but also mundane things like some instances of the common cold.
Does coronavirus cause death Statistically speaking, you have a 98% chance of survival, and 80% chance that you won’t need medical care at all even if you get infected, so stay positive, stay at home as much as possible, eat healthy, treat everyone as if they are infected, treat other as if you are infected. Many people get infected and get better without even knowing they had it.What can one do to prepare for the coronavirus? What should one buy or do regularly So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.What should I buy to prepare for a coronavirus epidemic if I had to stay in my home for a very long time So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.
What are the symptoms of Coronavirus The symptoms of this virus include high fever, respiratory problems, shortness of breath and coughing. In extreme cases, it can even lead to pneumonia, kidney failure and death.To prevent infection from spreading, one should wash hands regularly, cover mouth and nose while coughing and sneezing and eat properly cook meat and eggs."""

In [6]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [7]:
lemmer = nltk.stem.WordNetLemmatizer()

In [8]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [9]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [10]:
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [11]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "hi Babe", "how are you?")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me","Thanks for asking"]

In [12]:
def greeting(sentence):
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
      return random.choice(GREETING_RESPONSES)

In [13]:
sent_tokens

['How dangerous is the coronavirus Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.How has the coronavirus affected you Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly.',
 'Both in respect of career and in respect of mental health.',
 'Is there a treatment for the Coronavirus They are experimenting with various medicines and it seems some anti-malarial drugs show promise.',
 'However, they help only a little bit, as evidenced by the mortality rate in places like Italy or Spain.',
 'They are best if administered early, before the lungs are damaged.',
 'Is there a vaccine for the coronavirus We don’t “find” vaccines.',
 'That’s the language yellow media uses, but it’s far from the truth.',
 'We know the virus, and we know what it looks like.',
 'Now we need to produce a sufficient amount of attenuated virii.',
 'For that, we inject the virus into a foreign host, like a chicken 

In [31]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [32]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")


ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
'How dangerous is the coronavirus Covid-19
ROBO: How dangerous is the coronavirus Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.How has the coronavirus affected you Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly.
Is there a vaccine for the coronavirus
ROBO: Is there a vaccine for the coronavirus We don’t “find” vaccines.
what is chatbot
ROBO: I am sorry! I don't understand you
what is the symtom
ROBO: I am sorry! I don't understand you
what is corronavirus
ROBO: I am sorry! I don't understand you
what is corrona virus
ROBO: Most viruses will die, because the host is so different from humans, that it can not work.
Does coronavirus cause death Statistically
ROBO: Does coronavirus cause death Statistically speaking, you have a 98% chance of survival, and 80% chance that you won’t 

KeyboardInterrupt: ignored

In [21]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo-model-2020.03.19.tar.gz")

downloading: 100%|##########| 374434792/374434792 [00:22<00:00, 16410782.72B/s]


In [22]:
passage ="""How dangerous is the coronavirus?
Covid-19 is deadly, although fatality rates skyrocket for the elderly and those with compromised immune systems.

How has the coronavirus affected you?
Well if you ask me personally, the outbreak of novel Coronavirus, has affected me significantly. Both in respect of career and in respect of mental health.

Is there a treatment for the Coronavirus?
They are experimenting with various medicines and it seems some anti-malarial drugs show promise. However, they help only a little bit, as evidenced by the mortality rate in places like Italy or Spain. They are best if administered early, before the lungs are damaged.

Is there a vaccine for the coronavirus?
We don’t “find” vaccines. That’s the language yellow media uses, but it’s far from the truth.
We know the virus, and we know what it looks like. Now we need to produce a sufficient amount of attenuated virii. For that, we inject the virus into a foreign host, like a chicken embryo in an egg. Most viruses will die, because the host is so different from humans, that it can not work. Some will mutate to work, and those multiply.

How did coronaviruses get their name?
The coronavirus family was discovered first in 1960s but we don’t know where they come from. They get the name given their crown-like (“corona”) shape. Following a December 2019 outbreak in China, the World Health Organization identified a new type called 2019 novel coronavirus (2019-nCoV).

What are coronaviruses?
Coronavirus is a name for a group of viruses that target mammals, including cats, dogs, and humans. They’re well known for causing potentially fatal conditions like kidney failure, pneumonia but also mundane things like some instances of the common cold.


Does coronavirus cause death?
Statistically speaking, you have a 98% chance of survival, and 80% chance that you won’t need medical care at all even if you get infected, so stay positive, stay at home as much as possible, eat healthy, treat everyone as if they are infected, treat other as if you are infected. Many people get infected and get better without even knowing they had it.
 
What can one do to prepare for the coronavirus? What should one buy or do regularly?
So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.

What should I buy to prepare for a coronavirus epidemic if I had to stay in my home for a very long time?
So, one thing you might do before it hits your region is to buy enough food for your freezer as well as canned goods and stock it so that, when the Virus hits, you won’t be running out into a store crowded with infected people. When it hits, stay home for work and school and strictly avoid crowded places. Stock up on real bar soap (like Ivory Gold) and wash hands frequently. Avoid sanitizers. Try to avoid clinics or EDs unless your symptoms become an issue as you don’t want to spread the disease beyond your home. Stock up on jello, G-Zero, plain decongestants like Mucinex, Bouillon or clear soups, lots of Kleenex and avoid solids and dairy products while you are symptomatic. The rest is just common sense.

What are the symptoms of Coronavirus?
The symptoms of this virus include high fever, respiratory problems, shortness of breath and coughing. In extreme cases, it can even lead to pneumonia, kidney failure and death.
To prevent infection from spreading, one should wash hands regularly, cover mouth and nose while coughing and sneezing and eat properly cook meat and eggs."""

In [33]:
result=predictor.predict(passage=passage,question="How did coronaviruses get their name?")
result['best_span_str']

'crown-like (“corona”) shape'

In [34]:
result=predictor.predict(passage=passage,question="what is symptoms")
result['best_span_str']

'high fever, respiratory problems, shortness of breath and coughing'